# $L^2_\mu$ training of Fourier Neural Operator

In [1]:
# MIT License
# Copyright (c) 2025
#
# This is part of the dino_tutorial package
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND.
# For additional questions contact Thomas O'Leary-Roseberry

import os, sys
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

sys.path.append('../../')

from dinotorch_lite import *


## Load the Data

In [13]:
data_dir = 'data/full_state/'

mq_data_dict = np.load(data_dir+'mq_data.npz')

q_data = mq_data_dict['q_data']
m_data = mq_data_dict['m_data']
fno_metadata = np.load(data_dir+'fno_metadata.npz')

d2v = fno_metadata['d2v_param']
v2d = fno_metadata['v2d_param']
nx = fno_metadata['nx']
ny = fno_metadata['ny']

n_data, dQ = q_data.shape
n_data, dM = m_data.shape

print('dQ = ',dQ,', dM = ',dM)

m_train = torch.Tensor(m_data[:-800])
q_train = torch.Tensor(q_data[:-800])

m_test = torch.Tensor(m_data[-200:])
q_test = torch.Tensor(q_data[-200:])



# Set up datasets and loaders
l2train = L2Dataset(m_train,q_train)
l2test = L2Dataset(m_test,q_test)
batch_size = 32

train_loader = DataLoader(l2train, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(l2test, batch_size=batch_size, shuffle=True)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

dQ =  3322 , dM =  1661


In [24]:
model_settings = fno2d_settings(modes1=12, modes2=12, width=256, n_layers=4, d_out=2)
model = VectorFNO2D(v2d=[d2v, d2v], d2v=[v2d, v2d], nx=nx, ny=ny, dim=2, settings=model_settings).to(device) 

x = torch.Tensor(m_data[0:2])

x.shape

x.view(x.shape[0], 1, nx+1, ny+1)
# model(x)

RuntimeError: shape '[2, 1, 41, 21]' is invalid for input of size 3322

In [25]:
41*21

861

In [38]:
sys.path.append(os.environ.get('HIPPYLIB_PATH'))
import hippylib
from linear_elasticity_model import *

settings = linear_elasticity_settings()
model = linear_elasticity_model(settings)

Number of dofs: STATE=3322, PARAMETER=1661, ADJOINT=3322
Number of observation points: 50


In [39]:
settings['nx']

40

In [40]:
settings['ny']

20

## $L^2_\mu$ training

In [21]:
# model_settings = fno2d_settings(modes1=12, modes2=12, width=256, n_layers=4, d_out=2)
# model = VectorFNO2D(v2d=[d2v, d2v], d2v=[v2d, v2d], nx=nx, ny=ny, dim=2, settings=model_settings).to(device) 


# n_epochs = 100
# loss_func = normalized_f_mse
# lr_scheduler = None

# optimizer = torch.optim.Adam(model.parameters())

# network, history = l2_training(model,loss_func,train_loader, validation_loader,\
#                      optimizer,lr_scheduler=lr_scheduler,n_epochs = n_epochs)

# rel_error = evaluate_l2_error(model,validation_loader)

# print('L2 relative error = ', rel_error)

# torch.save(model.state_dict(), data_dir+'l2_model.pth')

IndexError: too many indices for tensor of dimension 1